# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#import current weather data from csv
clean_weather_data = pd.read_csv('../Output/clean_weather_data.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_weather_data[["Latitude", "Longitude"]]
humidity = clean_weather_data['Humidity (%)']

fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

### Narrowing Data to My Ideal Weather Conditions
My ideal weather conditions include:
- Anywhere between 75 and 80 degrees.
- A pleasent breeze (5 - 24 mph)
- Some clouds in the sky to admire (20% to 50%)

In [5]:
# narrowing the data to my ideal weather conditions
ideal_weather_data = clean_weather_data.loc[(clean_weather_data['Temperature (F)'] >= 75) & (clean_weather_data['Temperature (F)'] < 80),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Wind Speed (mph)'] >= 5) & (ideal_weather_data['Wind Speed (mph)'] < 24),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Cloudiness (%)'] >= 20) & (ideal_weather_data['Cloudiness (%)'] < 50),:]

ideal_weather_data = ideal_weather_data.set_index('City')
ideal_weather_data

,Latitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City,,,,,,,
bathsheba,13.2167,-59.5167,79.41,79.41,89,20,10.36
boguchany,58.3814,97.4531,76.46,76.46,41,20,5.14
loiza,18.4313,-65.8802,78.73,81.05,86,40,13.80
englewood,39.6478,-104.9878,77.79,84.51,27,40,5.75
axim,4.8699,-2.2405,75.58,75.58,86,35,8.46
oistins,13.0667,-59.5333,78.89,78.89,89,20,10.36
avera,33.1940,-82.5271,76.03,80.11,80,29,5.59
samarai,-10.6167,150.6667,79.11,79.11,86,43,15.86


### Find Hotels in My Ideal Weather Conditions

In [14]:
hotel_df = pd.DataFrame(columns = ['City','Hotel Name','Location', 'Rating'])
hotel_df = hotel_df.set_index('City')

In [15]:
# find the hotel closest to city's coordinates
search_radius= 5000
search_type = 'hotel'

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
query_perams = f'&radius={search_radius}&type={search_type}&key={g_key}'

# use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_data.iterrows():
  
    lat = row['Latitude']
    lng = row['Longitude']
        
    # get location type from df
    location = f'location={lat},{lng}'
    
    # assemble url and make API request
    print(f"Searching for Hotel in {index}.")
    query_url = base_url + location + query_perams
    response = requests.get(query_url).json()   
    
    if response['status'] == 'OK':
    
        # results code stem
        results = response['results'][1]
     
        try: 
            hotel_df.loc[index, 'Hotel Name'] = results['name']
            hotel_df.loc[index, 'Location'] = (results['plus_code']['compound_code'])[8:]
            hotel_df.loc[index, 'Rating'] = results['rating']
            hotel = results['name']
            print(f'Hotel: {hotel}')
        
        except (KeyError, IndexError):
            print("No Hotels Found Nearby... Skipping.")
    else:
        print("No Hotels Found Nearby... Skipping.")

    print("------------")

Searching for Hotel in bathsheba.
No Hotels Found Nearby... Skipping.
------------
Searching for Hotel in boguchany.
Hotel: Rayonnyy Dom Kul'tury
------------
Searching for Hotel in loiza.
Hotel: United States Postal Service
------------
Searching for Hotel in englewood.
Hotel: Walmart Supercenter
------------
Searching for Hotel in axim.
Hotel: Axim Beach Resort And Spa
------------
Searching for Hotel in oistins.
Hotel: Butterfly Beach Hotel
------------
Searching for Hotel in avera.
Hotel: Hiram Bobo III, PC
------------
Searching for Hotel in samarai.
Hotel: Nuli Sapi
------------


In [9]:
hotel_df = pd.merge(hotel_df,ideal_weather_data, on = ['City'], how='left')
hotel_df = hotel_df.dropna(how='any')
hotel_df

,Hotel Name,Location,Rating,Latitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City,,,,,,,,,,
boguchany,Rayonnyy Dom Kul'tury,"Boguchany, Krasnoyarsk Krai, Russia",4.3,58.3814,97.4531,76.46,76.46,41,20,5.14
loiza,United States Postal Service,"Loíza, Puerto Rico",2.9,18.4313,-65.8802,78.73,81.05,86,40,13.80
englewood,Walmart Supercenter,"Englewood, CO, USA",3.8,39.6478,-104.9878,77.79,84.51,27,40,5.75
axim,Axim Beach Resort And Spa,"Axim, Ghana",3.7,4.8699,-2.2405,75.58,75.58,86,35,8.46
oistins,Butterfly Beach Hotel,"Oistins, Barbados",4.3,13.0667,-59.5333,78.89,78.89,89,20,10.36
avera,"Hiram Bobo III, PC","Gibson, GA, USA",2.5,33.1940,-82.5271,76.03,80.11,80,29,5.59
samarai,Nuli Sapi,"Kasabanalua, Papua New Guinea",5,-10.6167,150.6667,79.11,79.11,86,43,15.86


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<c><dl>
<dt>Location:</dt><dd>{Location}</dd>
<dt>Hotel:</dt><dd>{Hotel Name}</dd>
<dd>Rated {Rating} out of 5</dd>
<dt>
</c></dl>
"""

In [11]:
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hover = hotel_df['Location']

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, hover_text=hover, label='', info_box_content=hotel_info, display_info_box=None)
fig1.add_layer(marker_layer)

## The only thing left is to book my flight!
Below are all the locations currently experiencing my ideal weather conditions. Click on each pin to view a hotel in the area.
Note: Weather is subject to change upon arrival.

In [16]:
# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))